In [28]:
"""
Link to the kaggle data: https://www.kaggle.com/datasets/robikscube/driving-video-with-object-tracking/data
Link to the data processing tutorial: https://www.kaggle.com/code/robikscube/working-with-video-in-python-youtube-tutorial
"""

'\nLink to the kaggle data: https://www.kaggle.com/datasets/robikscube/driving-video-with-object-tracking/data\nLink to the data processing tutorial: https://www.kaggle.com/code/robikscube/working-with-video-in-python-youtube-tutorial\n'

---
# Setting the enviroment for Colab

In [29]:
# # For Colab
# from google.colab import drive
# drive.mount('/content/drive')

# # For Colab:

# # Moves the project to /content, so accessing to the data is 20-30x times faster.
# !cp -r "/content/drive/MyDrive/RetinaNet" "/content/"

# # Change the current directory to the notebook folder:
# %cd /content/RetinaNet/Notebooks

# # List the files in the folder:
# !ls
# !ls ../Data

# # !pip uninstall -y jax jaxlib
# # !pip install -q "tensorflow==2.16.1" "keras==3.0.0" "keras-cv==0.9.0" "ml_dtypes>=0.5"

# Imports, Paths, Constants

In [30]:
from pathlib import Path
import sys
from pprint import pprint

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import IPython.display as ipd
import tensorflow as tf

# !pip install keras-cv

import keras_cv # new API of keras 3 compatible with multiple backends (TensorFlow, PyTorch, Jax)

In [31]:
print("TensorFlow:", tf.__version__)
print('Python:', sys.version)
print("KerasCV:", keras_cv.__version__)
print("Models:", dir(keras_cv.models))


TensorFlow: 2.20.0
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
KerasCV: 0.9.0
Models: ['BASNet', 'Backbone', 'CLIP', 'CSPDarkNetBackbone', 'CSPDarkNetLBackbone', 'CSPDarkNetMBackbone', 'CSPDarkNetSBackbone', 'CSPDarkNetTinyBackbone', 'CSPDarkNetXLBackbone', 'CenterPillarBackbone', 'DeepLabV3Plus', 'DenseNet121Backbone', 'DenseNet169Backbone', 'DenseNet201Backbone', 'DenseNetBackbone', 'EfficientNetLiteB0Backbone', 'EfficientNetLiteB1Backbone', 'EfficientNetLiteB2Backbone', 'EfficientNetLiteB3Backbone', 'EfficientNetLiteB4Backbone', 'EfficientNetLiteBackbone', 'EfficientNetV1B0Backbone', 'EfficientNetV1B1Backbone', 'EfficientNetV1B2Backbone', 'EfficientNetV1B3Backbone', 'EfficientNetV1B4Backbone', 'EfficientNetV1B5Backbone', 'EfficientNetV1B6Backbone', 'EfficientNetV1B7Backbone', 'EfficientNetV1Backbone', 'EfficientNetV2B0Backbone', 'EfficientNetV2B1Backbone', 'EfficientNetV2B2Backbone', 'EfficientNetV2B3Backbone', 'EfficientN

In [32]:
Path.cwd()

WindowsPath('c:/Proyectos/George Brown Projects/2 Semester/Deep Learning 2/RetinaDet - Video with obj traking/Notebooks')

Relative path where the data videos are stored:

In [33]:
# Regular slashes "/", Path adapt it to each OS.
RELAT_PATH_VIDS = Path("../Data/bdd100k_videos_train_00/bdd100k/videos/train")
RELAT_PATH_VIDS


WindowsPath('../Data/bdd100k_videos_train_00/bdd100k/videos/train')

In [34]:
data = pd.read_parquet("../Data/mot_labels.parquet")
display(ipd.Markdown('### Metadata of the whole 1000 videos:'))
data

### Metadata of the whole 1000 videos:

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo
0,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,00089537,car,False,True,False,825.173210,1003.094688,355.011547,418.198614,True
1,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,00089538,car,False,True,False,484.295612,700.461894,346.697460,424.849885,True
2,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,00089539,pedestrian,False,True,False,645.588915,663.879908,338.383372,358.337182,True
3,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,00089540,car,False,False,False,120.969977,192.471132,359.168591,409.053118,True
4,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,00089541,car,False,False,False,251.501155,315.519630,354.180139,400.739030,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890841,00adbb3f-7757d4ea-0000202.jpg,00adbb3f-7757d4ea,201,00070587,car,False,True,False,105.312994,169.282443,451.341401,500.152672,True
2890842,00adbb3f-7757d4ea-0000202.jpg,00adbb3f-7757d4ea,201,00070591,car,False,True,False,312.328249,368.244275,460.368229,490.457656,True
2890843,00adbb3f-7757d4ea-0000202.jpg,00adbb3f-7757d4ea,201,00070593,car,False,True,False,368.533042,414.700981,467.384253,491.059444,True
2890844,00adbb3f-7757d4ea-0000202.jpg,00adbb3f-7757d4ea,201,00070581,car,False,True,False,0.524781,111.778426,446.064140,500.641399,True


In [35]:
len(data.category.values)

2890846

---
# Useful functions from the Data Exploration notebook:

In [36]:
""" This function rebuilds the relative path to each video file given its name. """
def video_full_path(vid_name : str, format : str) -> Path:
    return (RELAT_PATH_VIDS / f"{vid_name}.{format}")

#---------------------------------------------------------------------------------------------------------

""" This function checks which videos are available in the videos data folder. """
def available_videos(folder : Path) -> list:
    return [p.name[:-4] for p in folder.iterdir() if p.is_file()] # [:-4] is to removes the .mov extension

#---------------------------------------------------------------------------------------------------------

""" Return a specific frame given the video name and the frame index. """
def get_frame( video_name : str, frame_index : int) -> np.ndarray:
    vid_path = video_full_path(video_name, 'mp4')
    capture = cv2.VideoCapture(vid_path)
    capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index) # Set the frames pointer to the desired frame index.
    _, img = capture.read()
    capture.release()
    return img

#---------------------------------------------------------------------------------------------------------

""" This function returns a list of the frames within a given range, from a video. """
def get_frames_in_range(vid_name : str, begin : int, end : int) -> list:
    vid_path = video_full_path(vid_name, 'mp4')
    capture = cv2.VideoCapture(vid_path)
    frames = []
    capture.set(cv2.CAP_PROP_POS_FRAMES, begin) # Set the frames pointer to the begin index.
    for _ in range(begin, end+1): # includes the first and last frame.
            ret, img = capture.read()
            frames.append(img)
            if not ret:
                print("End of video reached, range given exceeds video length.")
                break
    capture.release()
    return frames

#---------------------------------------------------------------------------------------------------------

""" This method is just for visualization. It does 3 things:
    1. Filters the data with all the missing annotations.
    2. Counts the number of videos with at least one missing annotation.
    3. Group by videoName, showing the number of missings and the corresponding frameIndexes. """

def miss_annotations_df(data : pd.DataFrame) -> None:

    # Data with missing annotations (in category).
    miss_annot = data[data.category.isna()]
    display(miss_annot)

    # Videos with missing annotations.
    miss_annot_vids = miss_annot.videoName.unique()
    display(ipd.Markdown(f'### There is {len(miss_annot_vids)} videos with missing annotations:'))

    # DataFrame missing annotations per video.
    records = []
    for vid_name, grp in miss_annot.groupby(['videoName']):
        amount_miss = len(grp)
        frame_index = grp['frameIndex'].values
        records.append({
            'videoName': vid_name,
            '#_missing': amount_miss,
            'frameIndex': frame_index
        })
    display(pd.DataFrame(records))

---
# Data Preprocessing:

In [37]:
# given the list of available videos, filter the dataframe to only include those videos
available_vids = available_videos(RELAT_PATH_VIDS)
data = data[data["videoName"].isin(available_vids)].reset_index(drop=True)
display(ipd.Markdown('### Actual Data Available:'))
data

### Actual Data Available:

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo
0,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000512,car,False,False,False,491.778291,717.090069,320.923788,512.147806,True
1,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000513,car,False,False,True,928.267898,1279.538106,298.475751,571.177829,True
2,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000514,car,False,True,False,816.027714,1029.699769,347.528868,474.734411,True
3,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000515,car,False,True,False,811.870670,948.221709,325.912240,408.221709,True
4,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000516,car,False,True,False,673.025404,751.177829,325.080831,367.482679,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24701,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095269,car,False,True,False,309.096210,339.119090,362.636929,398.093633,True
24702,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095268,car,False,False,False,666.485325,698.489124,363.695173,389.627681,True
24703,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095227,car,False,False,False,331.510791,426.474820,344.652312,411.798561,True
24704,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095223,car,True,True,False,561.616628,661.385681,349.943532,384.110855,True


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24706 entries, 0 to 24705
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  24706 non-null  object 
 1   videoName             24706 non-null  object 
 2   frameIndex            24706 non-null  int64  
 3   id                    24666 non-null  object 
 4   category              24666 non-null  object 
 5   attributes.crowd      24666 non-null  object 
 6   attributes.occluded   24666 non-null  object 
 7   attributes.truncated  24666 non-null  object 
 8   box2d.x1              24666 non-null  float64
 9   box2d.x2              24666 non-null  float64
 10  box2d.y1              24666 non-null  float64
 11  box2d.y2              24666 non-null  float64
 12  haveVideo             24706 non-null  bool   
dtypes: bool(1), float64(4), int64(1), object(7)
memory usage: 2.3+ MB


In [39]:
categ_count = data['category'].value_counts(dropna=False)

classes_df = pd.DataFrame({
    'Count': categ_count.values,
    'Percentage': ((categ_count / len(data)) * 100).round(2)
})

classes_df


,Count,Percentage
category,,
car,20409,82.61
pedestrian,2662,10.77
truck,740,3.00
bicycle,329,1.33
rider,302,1.22
motorcycle,128,0.52
other person,69,0.28
None,40,0.16
bus,27,0.11


In [40]:
""" This function gives us the approximate rate at which frames are labelled according to the metadata. """

""" Suppositions  needed for this function to work:
    1. The labelled frames are evenly distributed.
    2. the first and last labelled frames are the actual first and last frames of the video.
    3. CAP_PROP_FRAME_COUNT is not too far from the actual number of frames in the video. """

    # NOTE: After generating the video, the frames look a little behind. That means that maybe suposition 2 is not completely true.
    #       So maybe this behavior can be improved assuming the last labelled frame does not fall at the end, but a few frames before.
    #       Then, the rate would be (total_frames - 1 - n) / (labelled_frames - 1), for n between 0 and 5. Change that in the future
    #       and check if the resulting video looks better.

def labelled_frames_rate(vid_name : str) -> float:
    vid_path = video_full_path(vid_name, 'mp4')
    capture = cv2.VideoCapture(vid_path)

    total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    labelled_frames = data.query(f'videoName == "{vid_name}"')["frameIndex"].nunique()

    capture.release()

    # Avoid division by zero in the rare case a video has only 1 frame labelled:
    if labelled_frames <= 1:
        return 1

    # Instead of dividing N/M, we count the number of gaps between elements of the sequences, that is (N-1)/(M-1). In that way, the gaps of one sequence
    # will get alineated with respect to the other's sequence, leading to the boundaries of the sequences match, allowing the sequences to align correctly.
    return (total_frames - 1) / (labelled_frames -1)

first_video_name = data["videoName"].iloc[0]
labelled_frames_rate(first_video_name)

5.990049751243781

### Build the column with the actual frame index:

In [41]:
data = data.copy()
rates = {vid_n: labelled_frames_rate(vid_n) for vid_n in data.videoName.unique()}
data['VideoFrameIndex'] = (data['frameIndex'] * data['videoName'].map(rates) ).round().astype(int)

In [42]:
data

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,VideoFrameIndex
0,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000512,car,False,False,False,491.778291,717.090069,320.923788,512.147806,True,0
1,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000513,car,False,False,True,928.267898,1279.538106,298.475751,571.177829,True,0
2,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000514,car,False,True,False,816.027714,1029.699769,347.528868,474.734411,True,0
3,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000515,car,False,True,False,811.870670,948.221709,325.912240,408.221709,True,0
4,00a2e3ca-5c856cde-0000001.jpg,00a2e3ca-5c856cde,0,00000516,car,False,True,False,673.025404,751.177829,325.080831,367.482679,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24701,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095269,car,False,True,False,309.096210,339.119090,362.636929,398.093633,True,1208
24702,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095268,car,False,False,False,666.485325,698.489124,363.695173,389.627681,True,1208
24703,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095227,car,False,False,False,331.510791,426.474820,344.652312,411.798561,True,1208
24704,00a820ef-2b98dcf5-0000202.jpg,00a820ef-2b98dcf5,201,00095223,car,True,True,False,561.616628,661.385681,349.943532,384.110855,True,1208


### Drop frames without annotations. Frames in which no object is detected.

This has to be done after creating "VideoFrameIndex", since otherwise the labelled_frames_rate would be incorrect.

In [43]:
miss_annotations_df(data)

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,VideoFrameIndex
13455,00a0f008-3c67908e-0000173.jpg,00a0f008-3c67908e,172,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1035
13456,00a0f008-3c67908e-0000174.jpg,00a0f008-3c67908e,173,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1041
13457,00a0f008-3c67908e-0000175.jpg,00a0f008-3c67908e,174,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1047
13458,00a0f008-3c67908e-0000176.jpg,00a0f008-3c67908e,175,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1053
13459,00a0f008-3c67908e-0000177.jpg,00a0f008-3c67908e,176,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1059
13460,00a0f008-3c67908e-0000178.jpg,00a0f008-3c67908e,177,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1065
13461,00a0f008-3c67908e-0000179.jpg,00a0f008-3c67908e,178,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1071
13462,00a0f008-3c67908e-0000180.jpg,00a0f008-3c67908e,179,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1077
13463,00a0f008-3c67908e-0000181.jpg,00a0f008-3c67908e,180,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1083
13464,00a0f008-3c67908e-0000182.jpg,00a0f008-3c67908e,181,None,None,None,None,None,NaN,NaN,NaN,NaN,True,1089


### There is 2 videos with missing annotations:

,videoName,#_missing,frameIndex
0,"(00a0f008-3c67908e,)",30,"[172, 173, 174, 175, 176, 177, 178, 179, 180, ..."
1,"(00a9cd6b-b39be004,)",10,"[121, 122, 123, 124, 125, 126, 127, 128, 129, ..."


In [44]:
data = data.dropna(subset=["category"], inplace=False) # inplacer=False creates a copy of the df and returns the copy.
miss_annotations_df(data)
# GOLD RULE: Never use inplace=True in a function. Is less efficient and error-prone. (Given by Chat GPT)


,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,VideoFrameIndex


### There is 0 videos with missing annotations:

""


Since the classes are highly imbalanced, we will just keep the most common ones for the preliminary results:

In [45]:
print(f"Available Categories: {data.category.unique()}")
KEEP = {"car","pedestrian"} # Most common classes.
data = data[data["category"].isin(KEEP)].copy()
print(f"Selected Categories: {data.category.unique()}")
# NOTE: The idea is to include more classes as the model is improved.

Available Categories: ['car' 'pedestrian' 'bus' 'rider' 'bicycle' 'motorcycle' 'truck'
 'other person']
Selected Categories: ['car' 'pedestrian']


### Grouped Dataframe

In [46]:
""" This funtion builds a df in with each row has all the annotations corresponding to each frame. """
def build_annotations_per_frame(labels_df) -> pd.DataFrame:
    records = []
    for (vid, fi, name), grp in labels_df.groupby(["videoName","VideoFrameIndex","name"]):
        boxes = grp[["box2d.x1","box2d.y1","box2d.x2","box2d.y2"]].values.astype("float32")
        classes = grp["category"].values
        attrs = grp[["attributes.occluded","attributes.truncated","attributes.crowd"]].values
        records.append({
            "videoName": vid,
            "VideoFrameIndex": int(fi),
            "name": name,
            "boxes": boxes,        # (N,4) xyxy absolutos
            "classes": classes,    # (N,)
            "attributes": attrs    # (N,3)
        })
    return pd.DataFrame(records)


In [47]:
annot_per_frame_df = build_annotations_per_frame(data)
annot_per_frame_df

,videoName,VideoFrameIndex,name,boxes,classes,attributes
0,00a04f65-8c891f94,0,00a04f65-8c891f94-0000001.jpg,"[[491.7362, 369.0664, 529.68463, 397.75912], [...","[car, car, car, car]","[[False, False, False], [False, False, False],..."
1,00a04f65-8c891f94,6,00a04f65-8c891f94-0000002.jpg,"[[489.28406, 374.13394, 527.255, 400.35074], [...","[car, car, car, car]","[[False, False, False], [False, False, False],..."
2,00a04f65-8c891f94,12,00a04f65-8c891f94-0000003.jpg,"[[484.8354, 374.13394, 524.8254, 402.94232], [...","[car, car, car, pedestrian, pedestrian, car]","[[False, False, False], [False, False, False],..."
3,00a04f65-8c891f94,18,00a04f65-8c891f94-0000004.jpg,"[[480.3867, 374.13394, 522.39575, 405.53394], ...","[car, car, car, pedestrian, pedestrian, pedest...","[[False, False, False], [False, False, False],..."
4,00a04f65-8c891f94,24,00a04f65-8c891f94-0000005.jpg,"[[475.938, 374.13394, 519.9661, 408.12552], [3...","[car, car, car, pedestrian, car]","[[False, False, False], [False, False, False],..."
...,...,...,...,...,...,...
1978,00a9cd6b-b39be004,1187,00a9cd6b-b39be004-0000199.jpg,"[[714.2274, 341.10788, 748.863, 415.62683], [8...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1979,00a9cd6b-b39be004,1193,00a9cd6b-b39be004-0000200.jpg,"[[704.7813, 343.207, 739.41693, 417.72595], [1...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1980,00a9cd6b-b39be004,1199,00a9cd6b-b39be004-0000201.jpg,"[[694.2857, 341.10788, 728.92126, 415.62683], ...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1981,00a9cd6b-b39be004,1205,00a9cd6b-b39be004-0000202.jpg,"[[678.5423, 344.25656, 713.17786, 422.97375], ...","[pedestrian, car, car, car, car, car]","[[False, False, False], [False, False, False],..."


---
# Model Arquitecture

### Splits into train/val or train/test function

In [48]:
""" This function splits the dataset into 2 data sets, could be train and val, or train and test. """
def split_by_video(frames_df, val_ratio=0.2, seed=7) -> tuple[pd.DataFrame, pd.DataFrame]:
    videos = frames_df.videoName.unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(videos)
    n_val = int(len(videos)*val_ratio)
    val_vids = set(videos[:n_val])
    train_df = frames_df[~frames_df.videoName.isin(val_vids)].reset_index(drop=True)
    val_df   = frames_df[frames_df.videoName.isin(val_vids)].reset_index(drop=True)
    return train_df, val_df

train_df, val_df = split_by_video(annot_per_frame_df)
display(ipd.Markdown('### Metadata of the train set:'))
display(train_df)
print(f'Amount of videos in the train set: {len(train_df.videoName.unique())}')

display(ipd.Markdown('### Metadata of the val set:'))
display(val_df)
print(f'Amount of videos in the val set: {len(val_df.videoName.unique())}')


### Metadata of the train set:

,videoName,VideoFrameIndex,name,boxes,classes,attributes
0,00a04f65-af2ab984,0,00a04f65-af2ab984-0000001.jpg,"[[810.2078, 380.78522, 1047.9907, 522.9561], [...","[car, car, car, car, car, car, car, car, car, ...","[[False, False, False], [False, True, False], ..."
1,00a04f65-af2ab984,6,00a04f65-af2ab984-0000002.jpg,"[[813.5335, 378.7067, 1056.3048, 519.6305], [1...","[car, car, car, car, car, car, car, car, car, ...","[[False, False, False], [False, True, False], ..."
2,00a04f65-af2ab984,12,00a04f65-af2ab984-0000003.jpg,"[[816.85913, 376.62817, 1073.7644, 523.78754],...","[car, car, car, car, car, car, car, car, car, ...","[[False, False, False], [False, True, False], ..."
3,00a04f65-af2ab984,18,00a04f65-af2ab984-0000004.jpg,"[[821.0162, 377.87527, 1093.3025, 529.1917], [...","[car, car, car, car, car, car, car, car, car, ...","[[False, False, False], [False, True, False], ..."
4,00a04f65-af2ab984,24,00a04f65-af2ab984-0000005.jpg,"[[825.1732, 379.1224, 1112.8407, 534.5958], [0...","[car, car, car, car, car, car, car, car, car, ...","[[False, False, False], [False, True, False], ..."
...,...,...,...,...,...,...
1573,00a9cd6b-b39be004,1187,00a9cd6b-b39be004-0000199.jpg,"[[714.2274, 341.10788, 748.863, 415.62683], [8...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1574,00a9cd6b-b39be004,1193,00a9cd6b-b39be004-0000200.jpg,"[[704.7813, 343.207, 739.41693, 417.72595], [1...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1575,00a9cd6b-b39be004,1199,00a9cd6b-b39be004-0000201.jpg,"[[694.2857, 341.10788, 728.92126, 415.62683], ...","[pedestrian, car, car, car, car, car, car]","[[False, False, False], [True, True, False], [..."
1576,00a9cd6b-b39be004,1205,00a9cd6b-b39be004-0000202.jpg,"[[678.5423, 344.25656, 713.17786, 422.97375], ...","[pedestrian, car, car, car, car, car]","[[False, False, False], [False, False, False],..."


Amount of videos in the train set: 8


### Metadata of the val set:

,videoName,VideoFrameIndex,name,boxes,classes,attributes
0,00a04f65-8c891f94,0,00a04f65-8c891f94-0000001.jpg,"[[491.7362, 369.0664, 529.68463, 397.75912], [...","[car, car, car, car]","[[False, False, False], [False, False, False],..."
1,00a04f65-8c891f94,6,00a04f65-8c891f94-0000002.jpg,"[[489.28406, 374.13394, 527.255, 400.35074], [...","[car, car, car, car]","[[False, False, False], [False, False, False],..."
2,00a04f65-8c891f94,12,00a04f65-8c891f94-0000003.jpg,"[[484.8354, 374.13394, 524.8254, 402.94232], [...","[car, car, car, pedestrian, pedestrian, car]","[[False, False, False], [False, False, False],..."
3,00a04f65-8c891f94,18,00a04f65-8c891f94-0000004.jpg,"[[480.3867, 374.13394, 522.39575, 405.53394], ...","[car, car, car, pedestrian, pedestrian, pedest...","[[False, False, False], [False, False, False],..."
4,00a04f65-8c891f94,24,00a04f65-8c891f94-0000005.jpg,"[[475.938, 374.13394, 519.9661, 408.12552], [3...","[car, car, car, pedestrian, car]","[[False, False, False], [False, False, False],..."
...,...,...,...,...,...,...
400,00a820ef-2b98dcf5,1184,00a820ef-2b98dcf5-0000198.jpg,"[[224.10721, 359.49692, 322.18536, 412.95715],...","[car, car, car, car, car, car, car, car, car]","[[False, False, False], [True, False, False], ..."
401,00a820ef-2b98dcf5,1190,00a820ef-2b98dcf5-0000199.jpg,"[[221.4135, 359.6675, 320.78595, 414.1116], [4...","[car, car, car, car, car, car, car, car, car]","[[False, False, False], [True, False, False], ..."
402,00a820ef-2b98dcf5,1196,00a820ef-2b98dcf5-0000200.jpg,"[[220.93294, 358.25073, 320.215, 413.67868], [...","[car, car, car, car, car, car, car, car, car]","[[False, False, False], [True, False, False], ..."
403,00a820ef-2b98dcf5,1202,00a820ef-2b98dcf5-0000201.jpg,"[[218.83382, 358.95044, 320.215, 414.68884], [...","[car, car, car, car, car, car, car, car, car]","[[False, False, False], [True, False, False], ..."


Amount of videos in the val set: 2


### Extract frames from videos and save them in a folder:

In [22]:
""" This function exports the labelled frames to a folder, so we can train the model only with this labelled images. """
def export_labelled_frames(frame_df, out_dir_path) -> None:
    out_dir_path = Path(out_dir_path)
    out_dir_path.mkdir(parents=True, exist_ok=True)

    for _, row in frame_df.iterrows():
        img = get_frame(row.videoName, row.VideoFrameIndex)
        # saves frame as jpg: /out_dir_path/videoName/name.jpg
        vid_dir = out_dir_path / row.videoName
        vid_dir.mkdir(exist_ok=True)
        cv2.imwrite(str(vid_dir / row['name']), img)


In [23]:
TRAIN_FRAMES_PATH = Path(r"..\Data\bdd100k_videos_train_00\bdd100k\Extracted_Frames\Train_Labelled_Frames")
VAL_FRAMES_PATH = Path(r"..\Data\bdd100k_videos_train_00\bdd100k\Extracted_Frames\Val_Labelled_Frames")

In [24]:
""" Run once to creat the labelled frames folders. """
# export_labelled_frames(train_df, TRAIN_FRAMES_PATH)
# export_labelled_frames(val_df, VAL_FRAMES_PATH)

' Run once to creat the labelled frames folders. '

### Give an ID to each category and build the mapping dictionaries

In [25]:
CLASSES = sorted(data.category.unique())
class_to_id = {c:i for i,c in enumerate(CLASSES)}
id_to_class = {i:c for c,i in class_to_id.items()}
print(f'Sorted Categories: {CLASSES}')
display(f'Dictionary id to class:')
display(id_to_class)


Sorted Categories: ['car', 'pedestrian']


'Dictionary id to class:'

{0: 'car', 1: 'pedestrian'}

### Normalize boxes

In [26]:
""" Not needed anymore, was defined for EfficientDet. """

# WIDTH, HEIGHT = 1280, 720 # Frames dimensions.

# def normalize_boxes(boxes, width, height):
#     boxes[:, [0, 2]] /= width
#     boxes[:, [1, 3]] /= height
#     return boxes

' Not needed anymore, was defined for EfficientDet. '

### Data set generation "On-the-fly' - Dataset tf.data with Ragged Tensors

- RaggedTensors: These are tensors whose subtensors have variable length.  
- Ragged batching: Create batches using RaggetTensors instead of doing padding.  
- Each image has a variable number of boxes, then the components of the tensors will have different sizes.

In [27]:
BBOX_FORMAT = "xyxy" # KerasCV format (xmin, ymin, xmax, ymax), Important for augmentation using KerasCV (augmentation with bounding boxes).

def gen_samples(frame_df, img_root): # Generator function.
    for _, row in frame_df.iterrows():
        img_path = Path(img_root) / row.videoName / row['name']
        img = tf.io.decode_jpeg(tf.io.read_file(str(img_path)), channels=3)
        h = tf.shape(img)[0]; w = tf.shape(img)[1]

        boxes = tf.convert_to_tensor(row.boxes, dtype=tf.float32)
        classes = tf.convert_to_tensor([class_to_id[c] for c in row.classes], tf.int32) # Converts the list of classes into a list of ids. Then converts it into a tensor.

        dict = {
            "images": img,
            "bounding_boxes": {
                "boxes": boxes,
                "classes": classes
            }
        }
        yield dict


In [28]:
TRAIN_ROOT = Path("../Data/bdd100k_videos_train_00/bdd100k/Extracted_Frames/Train_Labelled_Frames")
VAL_ROOT   = Path("../Data/bdd100k_videos_train_00/bdd100k/Extracted_Frames/Val_Labelled_Frames")
BATCH_SIZE = 4

# 'output_sig' define's the form and structure THAT PRODUCES THE DATASET using the generator function:
#       Form: Shape of the tensor
#       Structure: dict, list, tuple...
#       Type: Data type on each entry
# 'specs' stands for 'specifications'
output_sig = {
    "images": tf.TensorSpec(shape=(None,None,3), dtype=tf.uint8),    # (height, width, channels)
    "bounding_boxes": {
        "boxes": tf.TensorSpec(shape=(None,4), dtype=tf.float32),    # (num_boxes, [xmin, ymin, xmax, ymax])
        "classes": tf.TensorSpec(shape=(None,), dtype=tf.int32)      # (num_boxes,)
    }
}

ds_train = tf.data.Dataset.from_generator(
    lambda: gen_samples(train_df, TRAIN_ROOT),
    output_signature=output_sig
)

ds_val = tf.data.Dataset.from_generator(
    lambda: gen_samples(val_df, VAL_ROOT),
    output_signature=output_sig
)

# Batching (ragged)
# Batches should be shuffle for the train set so there is no temporal pattern within the batch, since frames come from videos.
ds_train = ds_train.shuffle(512).ragged_batch(BATCH_SIZE)
# Val set should not be shuffled since we need reproducibility.
ds_val = ds_val.ragged_batch(BATCH_SIZE)

""" Example tensor:
ds_train tensor: ( (batch_size, height, width, 3),
                   (batch_size, num_boxes, 4),
                   (batch_size, num_boxes) )
"""
# (Note: "num_boxes" is inferred from the shape of the boxes tensor.)
#  bounding_boxes["boxes"].shape == (batch, num_boxes, 4).

' Example tensor:\nds_train tensor: ( (batch_size, height, width, 3),\n                   (batch_size, num_boxes, 4),\n                   (batch_size, num_boxes) )\n'

In [29]:
# Shapes example:
batch = next(iter(ds_train))
print(batch["bounding_boxes"]["boxes"].shape)
print(batch["bounding_boxes"]["classes"].shape)

(4, None, 4)
(4, None)


### Augmentation

- Here we use KerasCV for data augmentation, since the bounding boxes should be resized as well every time the geometry of the frame changes.
- The variable "augmenter" is a keras LAYER (obj type: Layer), so it is callable as a function.
- For geometric augmentations, KerasCV expects a fixed structure dictionary:

In [30]:
""" Expected structure: """
# {
#    "images": img,
#    "bounding_boxes": {
#          "boxes": boxes,
#          "classes": classes,
#          "confidence": confidences (optional),
#    }
# }
# (Note: Classes need to be included so that KerasCV can still link each box to its corresponding class.)
# (If for example, a box is cropped in the augmentation, this boxes with its corresponding class, should be removed.)

' Expected structure: '

- What is the return? The same dictionary, but with:
    - The augmented image.
    - Updated boxes.
    - Classes are identical, no changes.
    - Maintains the batch.

This is how the resulting data is access:

In [31]:
# augmented["images"]                   # ← augmented image.
# augmented["bounding_boxes"]["boxes"]  # ← updated boxes.


In [32]:
IMG_SIZE = (512, 512)

augmenter = keras_cv.layers.Augmenter([
    keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format=BBOX_FORMAT),
    keras_cv.layers.JitteredResize(
        target_size = IMG_SIZE,
        scale_factor = (0.75,1.25),
        bounding_box_format=BBOX_FORMAT
    )
])
# For val set, just resize without randomness.
val_resizer = keras_cv.layers.JitteredResize(
    target_size = IMG_SIZE,
    scale_factor = (1.0, 1.0),    # No randomness, no zoom, no jitter.
    bounding_box_format=BBOX_FORMAT,
)

# Apply augmentation just to the train data set.
ds_train = ds_train.map(lambda x: augmenter(x), num_parallel_calls=tf.data.AUTOTUNE)
# Since the images are resized in the augmentation, the val set should be resized as well.
ds_val = ds_val.map(lambda x: val_resizer(x), num_parallel_calls=tf.data.AUTOTUNE,)
# Prefetch at the end.
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
ds_val   = ds_val.prefetch(tf.data.AUTOTUNE)


---
# Model Definition

### Define Model - RetinaNet from KerasCV

RetinaNet es a complex model. With the method **from_preset()** from RetinaNet, there is no need to specify the backbone, the neck, config anchors, select wether or not to rescale, ... It is basically a short name that gets the full configuration.

In [33]:
# Models included by default with presets:
print(sorted(keras_cv.models.RetinaNet.presets.keys()))
# Looks like retinanet_resnet50_pascalvoc is the only one included by default for RetinaNet.


['csp_darknet_l', 'csp_darknet_l_imagenet', 'csp_darknet_m', 'csp_darknet_s', 'csp_darknet_tiny', 'csp_darknet_tiny_imagenet', 'csp_darknet_xl', 'densenet121', 'densenet121_imagenet', 'densenet169', 'densenet169_imagenet', 'densenet201', 'densenet201_imagenet', 'efficientnetlite_b0', 'efficientnetlite_b1', 'efficientnetlite_b2', 'efficientnetlite_b3', 'efficientnetlite_b4', 'efficientnetv1_b0', 'efficientnetv1_b1', 'efficientnetv1_b2', 'efficientnetv1_b3', 'efficientnetv1_b4', 'efficientnetv1_b5', 'efficientnetv1_b6', 'efficientnetv1_b7', 'efficientnetv2_b0', 'efficientnetv2_b0_imagenet', 'efficientnetv2_b1', 'efficientnetv2_b1_imagenet', 'efficientnetv2_b2', 'efficientnetv2_b2_imagenet', 'efficientnetv2_b3', 'efficientnetv2_l', 'efficientnetv2_m', 'efficientnetv2_s', 'efficientnetv2_s_imagenet', 'mobilenet_v3_large', 'mobilenet_v3_large_imagenet', 'mobilenet_v3_small', 'mobilenet_v3_small_imagenet', 'resnet101', 'resnet101_v2', 'resnet152', 'resnet152_v2', 'resnet18', 'resnet18_v2', '

**Note:** When num_classes is different from the number of classes with which the model was trained, KerasCV removes the head layer and adds a new one adapted for 2 classes output. This new head begins from scrath, that is, weights are randomly initialized.

In [ ]:
NUM_CLASSES = len(CLASSES)
""" Not working model. When changing the number of classes, the classification head is updated, but the label encoder stayed configured for 20 classes. Then, shape mismatch errors rise when training. """
# model = keras_cv.models.RetinaNet.from_preset(
#     "retinanet_resnet50_pascalvoc",             # RetinaNet with ResNet50 backbone and wieghts pretrained on Pascal VOC dataset.
#     num_classes=NUM_CLASSES,                    # Number of classes is not 20, thus a new head layer is added to the model.
#     bounding_box_format=BBOX_FORMAT,
# )


""" Working model. The idea here is that instead to just using 'preset' to load the complete model, we load the backbone separetly and then we add the head layer. """
backbone = keras_cv.models.ResNet50Backbone.from_preset("resnet50_imagenet")

model = keras_cv.models.RetinaNet(
    backbone=backbone,
    num_classes=NUM_CLASSES,           # 2
    bounding_box_format=BBOX_FORMAT,   # "xyxy"
)


In [35]:
model.backbone.summary()

Model: "res_net50_backbone"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │       0 │ -                    │
│ (InputLayer)        │ None, 3)          │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rescaling           │ (None, None,      │       0 │ input_layer[0][0]    │
│ (Rescaling)         │ None, 3)          │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │   9,408 │ rescaling[0][0]      │
│                     │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1_bn            │ (None, None,      │     256 │ conv1_conv[0][0]     │
│ (BatchNormalizatio… │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1_relu          │ (None, None,      │       0 │ conv1_bn[0][0]       │
│ (Activation)        │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ pool1_pool          │ (None, None,      │       0 │ conv1_relu[0][0]     │
│ (MaxPooling2D)      │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │   4,096 │ pool1_pool[0][0]     │
│ (Conv2D)            │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │     256 │ v2_stack_0_block1_1… │
│ (BatchNormalizatio… │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │       0 │ v2_stack_0_block1_1… │
│ (Activation)        │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │  36,864 │ v2_stack_0_block1_1… │
│ (Conv2D)            │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │     256 │ v2_stack_0_block1_2… │
│ (BatchNormalizatio… │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │       0 │ v2_stack_0_block1_2… │
│ (Activation)        │ None, 64)         │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │  16,384 │ pool1_pool[0][0]     │
│ (Conv2D)            │ None, 256)        │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │  16,384 │ v2_stack_0_block1_2… │
│ (Conv2D)            │ None, 256)        │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │   1,024 │ v2_stack_0_block1_0… │
│ (BatchNormalizatio… │ None, 256)        │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │   1,024 │ v2_stack_0_block1_3… │
│ (BatchNormalizatio… │ None, 256)        │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ v2_stack_0_block1_… │ (None, None,      │       0 │ v2_stack_0_block1_0

 Total params: 23,561,152 (89.88 MB)

 Trainable params: 23,508,032 (89.68 MB)

 Non-trainable params: 53,120 (207.50 KB)

### Compile Model

In [36]:
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
    metrics=[],
    run_eagerly=True   # 👈 esto # Given by GPT to fix a bug. Dont know what it is for.
)


---
# Training

In [37]:
EPOCHS_PHASE1 = 5           # Train head-only
EPOCHS_PHASE2 = 20          # Total epochs (head + backbone)

## Transfer learning

#### For the first 5 epochs, just the head and the neck (FPN) are trained:
(When the goal is just to train the head, the neck should be trained too. Otherwise, the model will have lest capacity to adapt to the new data).

In [38]:
for layer in model.layers: # Just a double check to set everything to trainable.
    layer.trainable = True

for layer in model.backbone.layers: # Freeze the backbone layers.
    layer.trainable = False

In [39]:
# Example of how the first batch looks like:
for batch in ds_train.take(1):
    print(batch["bounding_boxes"]["boxes"])
    print(batch["bounding_boxes"]["classes"])
    break


<tf.RaggedTensor [[], [[0.3675537, 411.716, 55.740723, 456.84415],
      [264.28876, 413.56314, 357.34448, 476.56805],
      [208.26843, 416.6559, 267.66876, 459.45032],
      [141.22577, 419.4478, 179.79675, 451.48645],
      [110.538574, 419.63092, 133.26306, 438.3048],
      [55.740723, 412.46033, 80.9104, 437.4418]]   ,
 [[133.64758, 392.0416, 229.88525, 449.7421],
  [363.8767, 377.16214, 392.80896, 425.8746],
  [46.88617, 380.7866, 88.375305, 419.27893],
  [105.40112, 388.77402, 150.55249, 437.67105],
  [84.581055, 387.61313, 122.0072, 430.7687]]  ,
 [[131.91095, 264.15588, 512.0, 473.61896],
  [438.8783, 264.9584, 512.0, 378.11667],
  [39.075867, 271.88947, 72.767456, 297.06003]]]>
<tf.RaggedTensor [[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]>


In [40]:
for batch in ds_train.take(1):
    print(batch["images"].shape)
    print(batch["bounding_boxes"]["boxes"].bounding_shape())
    print(batch["bounding_boxes"]["classes"].bounding_shape())
    break


(4, 512, 512, 3)
tf.Tensor([4 5 4], shape=(3,), dtype=int64)
tf.Tensor([4 5], shape=(2,), dtype=int64)


In [41]:
print(ds_train.element_spec)

{'images': TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), 'bounding_boxes': {'boxes': RaggedTensorSpec(TensorShape([None, None, 4]), tf.float32, 1, tf.int64), 'classes': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int64)}}


In [43]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "efficientdet_car_pedestrian.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3
    )
]

# Limit sizes while depuring.
# ds_train = ds_train.take(100) # Number of batches, since the dataset is build with batches.
# ds_val   = ds_val.take(50)

# Numberof batches processed per epoch.
steps_per_epoch = 50
validation_steps = 10

history_1 = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS_PHASE1,  # 5 epochs for the first training.
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1
)


Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1246s 25s/step - loss: 56.4013 - val_loss: 2.0177 - learning_rate: 0.0010
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1158s 23s/step - loss: 2.3275 - val_loss: 1.7102 - learning_rate: 0.0010
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1157s 23s/step - loss: 1.5301 - val_loss: 1.4541 - learning_rate: 0.0010
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1137s 23s/step - loss: 1.3642 - val_loss: 1.3893 - learning_rate: 0.0010
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1144s 23s/step - loss: 1.2061 - val_loss: 1.1982 - learning_rate: 0.0010


## Fine-tuning the model

#### On the next 15 epochs, just the last stage of the backbone will be trained (CONV5):

In [44]:
for layers in model.backbone.layers: # Unfreeze just the stage 5 layers.
    name = layers.name.lower()
    if 'v2_stack_3' in name:         # Name of the stage 5 layers.
        layers.trainable = True

In [45]:
""" The model must be recompiled before being fit again. """
learning_rate = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Re-compile.
model.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
    run_eagerly=True,   # 👈 esto # Given by GPT to fix a bug. Dont know what it is for.
    metrics=[]
)

In [ ]:
history_2 = model.fit(
    ds_train,
    validation_data=ds_val,
    initial_epoch=EPOCHS_PHASE1,  # Continue training from the previous epoch (5).
    epochs=EPOCHS_PHASE2,         # Continue training for another 15 epochs.
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1
)


Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
